In [1]:
import pandas as pd
import logging
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import streamlit as st

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OPTION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OPTION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OPTION\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Penggunaan CPU: 43.6%
Penggunaan Memori: 77.1%
Penggunaan Memori melebihi batas! Menghentikan proses...


## Load DATASET

In [1]:
def load_dataset(file_path):
    return pd.read_csv(file_path)

if __name__ == "__main__":
    # Path ke file dataset yang diunggah
    file_path = 'CISI_ALL.csv'
    
    # Memuat dataset
    data = load_dataset(file_path)
    
    # Menampilkan beberapa baris pertama dataset
    print(data.head())


NameError: name 'pd' is not defined

## PREPOCESING

In [3]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Unduh stopwords dan tokenizer NLTK jika belum tersedia
nltk.download('stopwords')
nltk.download('punkt')

# Load dataset (contoh membaca dari file CSV)
file_path = "CISI_ALL.csv"  # Ganti dengan path file Anda
data = pd.read_csv(file_path)

# Fungsi preprocessing
def preprocess_text(text):
    # 1. Case folding (mengubah teks menjadi huruf kecil)
    text = text.lower()
    
    # 2. Remove special characters, angka, dan tanda baca
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. Tokenisasi
    tokens = word_tokenize(text)
    
    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Gabungkan kembali menjadi teks
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

# Terapkan preprocessing pada kolom teks
data['Preprocessed_Query'] = data['Query'].apply(preprocess_text)

# Simpan hasil preprocessing ke file baru
output_file = "preprocessed_dataset.csv"
data.to_csv(output_file, index=False)

print(f"Preprocessing selesai. Dataset tersimpan di {output_file}.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OPTION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OPTION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Preprocessing selesai. Dataset tersimpan di preprocessed_dataset.csv.


In [4]:
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the cleaned text
data['Tokens'] = data['Preprocessed_Query'].apply(lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128))

# Display the result
print(data[['Preprocessed_Query', 'Tokens']].head())


                                  Preprocessed_Query  \
0  edit dewey decim classif comaromi jp present s...   
1  use made technic librari slater report analysi...   
2  two kind power essay bibliograph control wilso...   
3  system analysi univers librari final report re...   
4  librari manag game report research project bro...   

                                              Tokens  
0  [101, 10086, 20309, 11703, 5714, 2465, 10128, ...  
1  [101, 2224, 2081, 6627, 8713, 5622, 10024, 308...  
2  [101, 2048, 2785, 2373, 9491, 12170, 16558, 36...  
3  [101, 2291, 20302, 7274, 2072, 4895, 16402, 20...  
4  [101, 5622, 10024, 3089, 24951, 2290, 2208, 31...  


In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['Tokens'], data['ID'], test_size=0.2, random_state=42
)

# Display the split dataset
print("Train texts:", train_texts.head())
print("Test texts:", test_texts.head())
print("Train labels:", train_labels.head())
print("Test labels:", test_labels.head())



Train texts: 254     [101, 15535, 3191, 6090, 2669, 17294, 1038, 22...
1066    [101, 2715, 2291, 2470, 5248, 7155, 3120, 8654...
638     [101, 3465, 3466, 23408, 4747, 4904, 12367, 22...
799     [101, 3930, 21183, 4014, 27885, 19454, 2229, 2...
380     [101, 9345, 7630, 12367, 14262, 7903, 4031, 23...
Name: Tokens, dtype: object
Test texts: 892     [101, 10424, 2063, 4226, 12273, 4487, 3367, 30...
1105    [101, 1041, 26989, 6895, 12367, 14262, 7903, 2...
413     [101, 2010, 29469, 5622, 10024, 3089, 2530, 20...
522     [101, 3465, 4842, 14192, 2006, 4115, 2489, 182...
1036    [101, 2646, 5248, 14833, 3089, 4012, 23041, 93...
Name: Tokens, dtype: object
Train labels: 254      255
1066    1067
638      639
799      800
380      381
Name: ID, dtype: int64
Test labels: 892      893
1105    1106
413      414
522      523
1036    1037
Name: ID, dtype: int64


In [6]:
import torch

# Convert to PyTorch tensors
train_inputs = torch.tensor(train_texts.tolist())
train_labels = torch.tensor(train_labels.tolist())
test_inputs = torch.tensor(test_texts.tolist())
test_labels = torch.tensor(test_labels.tolist())

# Display the results
print("Train inputs:", train_inputs)
print("Train labels:", train_labels)
print("Test inputs:", test_inputs)
print("Test labels:", test_labels)


Train inputs: tensor([[  101, 15535,  3191,  ...,     0,     0,     0],
        [  101,  2715,  2291,  ...,     0,     0,     0],
        [  101,  3465,  3466,  ...,     0,     0,     0],
        ...,
        [  101,  4012, 23041,  ...,     0,     0,     0],
        [  101, 28667,  2239,  ...,     0,     0,     0],
        [  101,  9345,  7630,  ...,     0,     0,     0]])
Train labels: tensor([ 255, 1067,  639,  ..., 1295,  861, 1127])
Test inputs: tensor([[  101, 10424,  2063,  ...,     0,     0,     0],
        [  101,  1041, 26989,  ...,     0,     0,     0],
        [  101,  2010, 29469,  ...,     0,     0,     0],
        ...,
        [  101,  5082,  6254,  ...,  4765, 10128,   102],
        [  101,  2128, 20414,  ...,     0,     0,     0],
        [  101,  4503,  3465,  ...,     0,     0,     0]])
Test labels: tensor([ 893, 1106,  414,  523, 1037,  615,  219, 1161,  650,  888,  577, 1253,
        1062,  568, 1109, 1114,  169, 1103, 1121,   68, 1041,  454,  671, 1095,
         19

In [1]:
from transformers import BertForSequenceClassification, AdamW

# Load the pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['ID'].unique()))

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop (simplified)
model.train()
for epoch in range(3):  # Number of epochs
    optimizer.zero_grad()
    outputs = model(train_inputs, labels=train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()


: 